In [4]:
import numpy as np
import pandas as pd
from sklearn import datasets, preprocessing
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, tree, ensemble

label_encoder = preprocessing.LabelEncoder()
iris = pd.read_csv("C:/Users/dell/Documents/iris.csv")

#x = iris.data
#y = iris.target
x=iris[['petal.length', 'petal.width', 'sepal.length', 'sepal.width']].values.T
iris['species'] = label_encoder.fit_transform(iris['species'])
y =iris[['species']].values.T
print(y.shape)


def initilize_parameters(n_x, n_h, n_y):
    np.random.seed(2)
    
    weight1 = np.random.randn(n_h, n_x)* 0.01
    weight2 = np.random.randn(n_y, n_h)* 0.01
    bias1 = np.zeros((n_h, 1), dtype= float)
    bias2 = np.zeros((n_y,1), dtype=float)
    
    assert (weight1.shape == (n_h, n_x))
    assert (bias1.shape == (n_h, 1))
    assert (weight2.shape == (n_y, n_h))
    assert (bias2.shape == (n_y, 1))
    
    params = {"w1" : weight1,
              "b1" : bias1,
              "w2" : weight2,
              "b2" : bias2}
    return params

def layer_sizes(x, y):
    n_x = x.shape[0]
    n_h = 6
    n_y = y.shape[0]
    return (n_x, n_y, n_h)





(1, 150)


In [7]:
def forward_propagation(x, params):
    # initilizing parameters from the obove dictionary
    w1 = params['w1']
    w2 = params['w2']
    b1 = params['b1']
    b2 = params['b2']
    
    # implement forward propagation to calculate probability
    z1 = np.dot(w1, x)+b1
    a1 = np.tanh(z1)       # tanh activation function
    z2 = np.dot(w2, a1)+b2
    a2 = 1/(1+np.exp(-z2)) # sigmoid activation function
    
    out = {"z1" : z1,
           "a1" : a1,
           "z2" : z2,
           "a2" : a2}
    return a2, out

def calculate_cost(a2, y, params):
    m = y.shape[1] #number of training examples
    
    w1 = params['w1']
    w2 = params['w2']
    
    cross_entropy_cost = np.multiply(np.log(a2), y) + np.multiply((1-y), np.log(1-a2))
    cost = np.sum(cross_entropy_cost)/m
    
    return cost


def backpropagation(params, out, x, y):
    m = x.shape[1] #no of training examples
    
    w1 = params['w1']
    w2 = params['w2']
    a1 =    out['a1']
    a2 =    out['a2']
    
    dz2 = a2-y
    dw2 = (1/m) * np.dot(dz2, a1.T)
    db2 = (1/m) * np.sum(dz2, axis=1, keepdims=True)
    dz1 = np.multiply(np.dot(w2.T, dz2), 1-np.power(a1,2))
    dw1 = (1/m) * np.dot(dz1, x.T)
    db1 = (1/m) * np.sum(dz1, axis=1, keepdims=True)
    
    grads = {"dw1" : dw1,
             "db1" : db1,
             "dw2" : dw2,
             "db2" : db2}
    return grads

def update_params(params, grads, lr=1.2):
    w1 = params['w1']
    w2 = params['w2']
    b1 = params['b1']
    b2 = params['b2']
    
    dw1 = grads['dw1']
    dw2 = grads['dw2']
    db1 = grads['db1']
    db2 = grads['db2']
    
    #update each parameters
    w1 = w1 - (lr * dw1)
    w2 = w2 - (lr * dw2)
    b1 = b1 - (lr * db1)
    b2 = b2 - (lr * db2)
    
    params = {        "w1" : w1,
                      "b1" : b1,
                      "w2" : w2,
                      "b2" : b2}
    return params



def NN_model(x, y, n_h, epoch=10000, print_cost=False):
    np.random.seed(3)
    n_x = layer_sizes(x, y)[0]
    n_y = layer_sizes(x, y)[2]
    
    params = initilize_parameters(n_x, n_h, n_y)
    w1 = params['w1']
    w2 = params['w2']
    b1 = params['b1']
    b2 = params['b2']
    
    for i in range(0, epoch):
        a2, out = forward_propagation(x,params)
        cost = calculate_cost(a2, y, params)
        grads = backpropagation(params, out, x, y)
        params = update_params(params, grads)
        
        if(print_cost and i%1000) == 0:
            print("cost after iteration {}: {}".format(i, cost))
            print("accuracy after iteration {}: {}".format(i, 1+cost))
    return params, n_h

params = NN_model(x, y, n_h=6, epoch=10000, print_cost=True)

cost after iteration 0: -4.160196845474832
accuracy after iteration 0: -3.1601968454748324
cost after iteration 1000: -0.0005775036881649005
accuracy after iteration 1000: 0.9994224963118351
cost after iteration 2000: -0.00031983391784820017
accuracy after iteration 2000: 0.9996801660821518
cost after iteration 3000: -0.00022142632689034752
accuracy after iteration 3000: 0.9997785736731096
cost after iteration 4000: -0.0001694571098698816
accuracy after iteration 4000: 0.9998305428901301
cost after iteration 5000: -0.00013730170146194118
accuracy after iteration 5000: 0.999862698298538
cost after iteration 6000: -0.0001154250550841122
accuracy after iteration 6000: 0.9998845749449159
cost after iteration 7000: -9.956854010927903e-05
accuracy after iteration 7000: 0.9999004314598907
cost after iteration 8000: -8.754340837640483e-05
accuracy after iteration 8000: 0.9999124565916236
cost after iteration 9000: -7.810876446607533e-05
accuracy after iteration 9000: 0.9999218912355339


In [3]:
def predict(params, x):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m)
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    ### START CODE HERE ### (≈ 2 lines of code)
    a2, out = forward_propagation(x, params)
    predictions = a2 > 0.5
    ### END CODE HERE ###
    
    return predictions

predictions = predict(params, x)
print ('Accuracy: %d' % float((np.dot(y,predictions.T) + np.dot(1-y,1-predictions.T))/float(y.size)*100) + '%')

TypeError: tuple indices must be integers or slices, not str